In [3]:
import numpy as np
import keras.backend as K
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input, Conv2D, Concatenate, GlobalAveragePooling2D, Flatten, Dense, Input
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical

In [4]:
train_data_path = './MVSA_Single/train'
val_data_path = './MVSA_Single/val'
test_data_path = './MVSA_Single/test'
weights_path = './weight/vgg16-places365_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [5]:
import tensorflow as tf

class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

    def reset_states(self):
        self.precision.reset_state()
        self.recall.reset_state()

f1_score = F1Score()

In [6]:
from keras.callbacks import ModelCheckpoint

checkpoint_filepath = 'best_scene_object_model.h5'
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_loss',  # Save the model when the validation loss is minimized
    save_best_only=True,  # Save only the best model
    mode='min',  # Minimize the validation loss
    verbose=1  # Print a message when the best model is saved
)

In [7]:
# Set the input size of the VGG16 model
input_shape = (224, 224, 3)

input_tensor = Input(shape=input_shape)

# Load the pre-trained VGG16 models
object_vgg16 = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)

object_vgg16.layers.pop()

# Freeze the convolutional layers, can not directly use, since all layer names should be unique.
for layer in object_vgg16.layers:
    layer.trainable = False
    layer._name = 'object_detection_' + layer.name

scene_model = VGG16(weights=None, include_top=False, input_tensor=input_tensor, classes=365)
scene_model.load_weights(weights_path)

scene_model.layers.pop()

# Freeze the convolutional layers
for layer in scene_model.layers:
    layer.trainable = False

# Combine the output of both models    
combined_output = Concatenate()([object_vgg16.output, scene_model.output])

# Add a GlobalAveragePooling2D layer to perform spatial average pooling
pooled_output = GlobalAveragePooling2D()(combined_output)

# Add a new final layer to the combined model
final_output = Dense(1024, activation='relu')(pooled_output)
final_output = Dropout(0.5)(final_output)
final_output = Dense(512, activation='relu')(final_output)
final_output = Dropout(0.5)(final_output)
final_output_1 = Dense(256, activation='relu')(final_output)
final_output_2 = Dropout(0.5)(final_output_1)
final_output_class = Dense(3, activation='softmax')(final_output_2)

# Create the final model
final_model = Model(inputs= input_tensor, outputs=final_output_class)

final_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[f1_score,'accuracy'])

58889256/58889256 [==============================] - 42s 1us/step


In [8]:
# Set up data generators for data augmentation and feeding data to the model
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        train_data_path,
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical')

val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(
        val_data_path,
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
        test_data_path,
        target_size=input_shape[:2],
        batch_size=32,
        class_mode='categorical')

Found 2415 images belonging to 3 classes.
Found 303 images belonging to 3 classes.
Found 480 images belonging to 3 classes.


In [9]:
#train_labels = to_categorical(train_generator.labels, num_classes=3)

# Train the model
final_model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // 32,
        epochs=100,
        validation_data=val_generator,
        validation_steps=800 // 32,
        callbacks=[checkpoint_callback])

# Evaluate the model on the test set
# test_loss, test_acc = final_model.evaluate_generator(test_generator, steps=50)
# print('Test accuracy:', test_acc)

C:\Users\Jeffan\AppData\Local\Temp\ipykernel_16796\763344743.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  final_model.fit_generator(


Epoch 1/50
62/62 [==============================] - ETA: 0s - loss: 1.2222 - f1_score: 0.2424 - accuracy: 0.3701

c:\Users\Jeffan\.conda\envs\capstone\lib\site-packages\keras\engine\training.py:2416: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()



Epoch 1: val_loss improved from inf to 1.06908, saving model to best_scene_object_model.h5
62/62 [==============================] - 422s 7s/step - loss: 1.2222 - f1_score: 0.2424 - accuracy: 0.3701 - val_loss: 1.0691 - val_f1_score: 0.0000e+00 - val_accuracy: 0.3828
Epoch 2/50
62/62 [==============================] - 371s 6s/step - loss: 1.0955 - f1_score: 0.1732 - accuracy: 0.4093
Epoch 3/50
62/62 [==============================] - 376s 6s/step - loss: 1.0816 - f1_score: 0.1895 - accuracy: 0.4148
Epoch 4/50
62/62 [==============================] - 378s 6s/step - loss: 1.0553 - f1_score: 0.1788 - accuracy: 0.4433
Epoch 5/50
62/62 [==============================] - 383s 6s/step - loss: 1.0470 - f1_score: 0.2177 - accuracy: 0.4560
Epoch 6/50
62/62 [==============================] - 380s 6s/step - loss: 1.0393 - f1_score: 0.2412 - accuracy: 0.4560
Epoch 7/50
62/62 [==============================] - 380s 6s/step - loss: 1.0174 - f1_score: 0.2903 - accuracy: 0.4936
Epoch 8/50
62/62 [======

In [18]:
test_loss, test_f1_score= final_model.evaluate_generator(test_generator, steps=50)
# print('Test accuracy:', test_accuracy)
print('Test f1_score:', test_f1_score)

C:\Users\Jeffan\AppData\Local\Temp\ipykernel_16796\110421484.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  test_loss, test_f1_score= final_model.evaluate_generator(test_generator, steps=50)


Test f1_score: 0.5145833492279053


In [10]:
from keras.models import load_model

custom_objects = {'F1Score': F1Score}
best_model = load_model('best_scene_object_model.h5', custom_objects=custom_objects)
test_loss, test_accuracy, test_f1_score = best_model.evaluate_generator(test_generator, steps=50)
print('Test accuracy:', test_accuracy)
print('Test f1_score:', test_f1_score)

C:\Users\Jeffan\AppData\Local\Temp\ipykernel_16796\2189405829.py:5: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  test_loss, test_accuracy, test_f1_score = best_model.evaluate_generator(test_generator, steps=50)


Test accuracy: 0.0041580041870474815
Test f1_score: 0.2541666626930237


In [11]:
from sklearn.metrics import f1_score

predict_result = np.argmax(final_model.predict_generator(test_generator), axis=-1)
actual_result = test_generator.classes

f1_score(actual_result, predict_result, average='micro')

C:\Users\Jeffan\AppData\Local\Temp\ipykernel_16796\580406551.py:3: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predict_result = np.argmax(final_model.predict_generator(test_generator), axis=-1)


0.26875

In [12]:
f1_score(actual_result, predict_result, average=None)

array([0.3356974 , 0.17171717, 0.24188791])

In [13]:
# Set the input size of the VGG16 model
input_shape = (224, 224, 3)

input_tensor = Input(shape=input_shape)

# Load the pre-trained VGG16 models
object_vgg16 = VGG16(weights='imagenet', include_top=False, input_tensor=input_tensor)

# Freeze the convolutional layers, can not directly use, since all layer names should be unique.
for layer in object_vgg16.layers:
    layer.trainable = False
    layer._name = 'object_detection_' + layer.name

scene_model = VGG16(weights=None, include_top=False, input_tensor=input_tensor, classes=365)
scene_model.load_weights(weights_path)

# Freeze the convolutional layers
for layer in scene_model.layers:
    layer.trainable = False

# Combine the output of both models    
combined_output = Concatenate()([object_vgg16.output, scene_model.output])

# Add a Flatten layer
pooled_output = Flatten()(combined_output)

# Add a new final layer to the combined model
final_output_1 = Dense(256, activation='relu')(pooled_output)
final_output_2 = Dropout(0.5)(final_output_1)
final_output_class = Dense(3, activation='softmax')(final_output_2)

# Create the final model
final_model = Model(inputs= input_tensor, outputs=final_output_class)

final_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])

# Train the model
final_model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // 32,
        epochs=50,
        validation_data=val_generator,
        validation_steps=800 // 32)

# Evaluate the model on the test set
test_loss, test_acc = final_model.evaluate_generator(test_generator, steps=50)
print('Test accuracy:', test_acc)

c:\Users\Jeffan\.conda\envs\capstone\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
C:\Users\Jeffan\AppData\Local\Temp\ipykernel_16796\1876040750.py:38: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  final_model.fit_generator(


Epoch 1/50
62/62 [==============================] - 489s 8s/step - loss: 1.1617 - accuracy: 0.4078 - val_loss: 1.0375 - val_accuracy: 0.4851
Epoch 2/50
62/62 [==============================] - 387s 6s/step - loss: 1.0222 - accuracy: 0.4738
Epoch 3/50
62/62 [==============================] - 352s 6s/step - loss: 1.0087 - accuracy: 0.4916
Epoch 4/50
62/62 [==============================] - 396s 6s/step - loss: 0.9744 - accuracy: 0.5099
Epoch 5/50
62/62 [==============================] - 373s 6s/step - loss: 0.9485 - accuracy: 0.5430
Epoch 6/50
62/62 [==============================] - 403s 6s/step - loss: 0.9390 - accuracy: 0.5536
Epoch 7/50
62/62 [==============================] - 391s 6s/step - loss: 0.9117 - accuracy: 0.5694
Epoch 8/50
62/62 [==============================] - 366s 6s/step - loss: 0.8873 - accuracy: 0.5765
Epoch 9/50
62/62 [==============================] - 341s 5s/step - loss: 0.8891 - accuracy: 0.5704
Epoch 10/50
62/62 [==============================] - 340s 5s/step -

KeyboardInterrupt: 

In [ ]:
predict_result = np.argmax(final_model.predict_generator(test_generator), axis=-1)
actual_result = test_generator.classes

f1_score(actual_result, predict_result, average='micro')

<ipython-input-19-fd4bacd1307b>:1: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predict_result = np.argmax(final_model.predict_generator(test_generator), axis=-1)


0.45748116254036597

In [ ]:
f1_score(actual_result, predict_result, average=None)

array([0.24145786, 0.1503268 , 0.59478886])